### **Install Natural Language ToolKit (NLTK) module (and some other modules)**


In [2]:
#the module 'sys' allows istalling module from inside Jupyter
import sys

!{sys.executable} -m pip install numpy
import numpy as np

!{sys.executable} -m pip install pandas
import pandas as pd

#Natrual Language ToolKit (NLTK)
!{sys.executable} -m pip install nltk
import nltk

!{sys.executable} -m pip install sklearn
from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer #bag-of-words vectorizer
from sklearn.decomposition import LatentDirichletAllocation #package for LDA

# Plotting tools

from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.lda_model

import matplotlib.pyplot as plt
%matplotlib inline

#define text normalization function
%run ./Text_Normalization_Function.ipynb #defining text normalization function

#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### **Read the CSV file**

In [3]:
# Now use pandas to read the csv file
intern_survey = pd.read_csv("/content/Intern Surveys to Inter 2018-2021.csv")

In [4]:
# Read open-ended questions
challenges = intern_survey["Challenges"]
virtual_work_impacting_on_performance = intern_survey["Virtual work impacting on performance"]

### **Open-ened Question 1: Challenges**

In [12]:
# Change 'business_area_goals_and_work' into a list of strings
STR_virtual_work_impacting_on_performance = []
for item in virtual_work_impacting_on_performance:
    if isinstance(item, float):
        item = str(item)
    STR_virtual_work_impacting_on_performance.append(item)

In [13]:
# Eliminate missing values
STR_virtual_work_impacting_on_performance = [item for item in STR_virtual_work_impacting_on_performance if item != 'nan']
STR_virtual_work_impacting_on_performance

["Absolutely. I know it is an unfortunate situation and a lot of the aspects that make the internship as strong as it is can't be replicated remotely. \n\nI think working in the research department though I'd hoped to get to work on a challenging project that challenged my economic skills. But, I also understand this is hard for managers to provide in the WFH environment because it is harder to get over that learning curve remotely.  ",
 'As I stated earlier, there are a lot of distractions when working from home. However, I have been able to overcome these distractions with time and focus on assignments. ',
 'As mentioned above, it can feel a bit isolating. I also feel like my managers are busy people so I often am waiting for higher-ups to get back to me. ',
 "As stated above, technology-wise it does present some challenges and has taken up a bit more time than expected. In addition, being at home with my family can be difficult as my work day isn't always entirely uninterrupted, so 

## **Define two functions what will display the results of fitting a topic model.**

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

## **Text Normalization & Vectorization**

By normalizing text, we can elmininate stopwords, extend contractions, do stemming or lemmatization.

In [14]:
# Normalize text
NORM_Q2 = normalize_corpus(STR_virtual_work_impacting_on_performance)

Since for topic modeling we need text data in the Bag-of-Words representation, let's vectorize our normalized_toy_corpus and call it bow_toy_corpus:

In [15]:
#define the bag-of-words vectorizer:
bow_vectorizer = CountVectorizer()

#vectorize the normalized data:
bow_Q2 = bow_vectorizer.fit_transform(NORM_Q2)

Have a look at the Bag-of-Words representation of our corpus

In [16]:
df = pd.DataFrame(data = bow_Q2.todense(), columns = bow_vectorizer.get_feature_names_out())

In [17]:
# Assuming 'df' is your DataFrame
column_sums = df.sum()
sorted_sums = column_sums.sort_values(ascending=False)
sorted_sums

work       37
think      19
people     17
time       15
person     15
           ..
human       1
huge        1
http        1
however     1
zone        1
Length: 423, dtype: int64

## **Topic Model (via Latent Dirichlet Allocation) on Q1 Data**

We will be using the LatentDirichletAllocation function which we already imported earlier (see installation of toolkit). The function has the following parameters to be set:

1. Number of topics to model: n_components
2. Parameter vector for the Dirichlet distribution for topics in a document: doc_topic_prior
3. Parameter vector for the Dirichlet distribution for words in a topic: topic_word_prior

Now, let's set the parameters and estimate the topic model:


In [ ]:
lda_Q2_corpus = LatentDirichletAllocation(n_components=4, max_iter=500,
                                           doc_topic_prior = 0.25,
                                           topic_word_prior = 0.25).fit(bow_Q2)

Display results by showing 10 most frequent (top) words for each topic

In [ ]:
no_top_words = 10
display_topics(lda_Q2_corpus, bow_vectorizer.get_feature_names_out(), no_top_words)

Topic 0:
work time feel able office virtual network person easy still
Topic 1:
work hard think virtually bank difficult home internship thing know
Topic 2:
person think day people work focus time bit help easy
Topic 3:
people feel virtual focus hard work meeting experience home person


Let's look at the relative frequency of the word of topics in a descending order.

In [ ]:
word_weights = lda_Q2_corpus.components_ / lda_Q2_corpus.components_.sum(axis=1)[:, np.newaxis]
word_weights_df = pd.DataFrame(word_weights.T,
                               index = bow_vectorizer.get_feature_names_out(),
                               columns = ["Topic_" + str(i) for i in range(4)])
word_weights_df.sort_values(by='Topic_0',ascending=False).head(10)

,Topic_0,Topic_1,Topic_2,Topic_3
work,0.051770,0.027750,0.018452,0.015316
time,0.020531,0.007575,0.013420,0.004468
feel,0.017676,0.010951,0.003165,0.015441
able,0.014847,0.000843,0.003181,0.008167
office,0.014844,0.000843,0.005741,0.008159
virtual,0.014839,0.004192,0.008295,0.015439
network,0.012027,0.000843,0.003180,0.000907
person,0.012024,0.004148,0.018553,0.011777
easy,0.012014,0.000843,0.010849,0.000907
still,0.012006,0.004214,0.003194,0.004537


## **Topic Modeling Visualization**

We can visualize the topics: topic size, frequency of words in a topic and so on. In this visualization, we can rank words in a topic by relevancy: Relevancy parameter is λ (0 ≤ λ ≤ 1).

*   small λ highlights potentially rare, but exclusive terms for the selected topic;
*   large values of λ (near 1) highlight frequent, but not necessarily exclusive, terms for the selected topic;

Relevancy = λ log[p(term | topic)] + (1 - λ) log[p(term | topic)/p(term)],

We installed all the pyLDAvis module required for this visualization in Toolkit. Not let's use it:


In [ ]:
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()

#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.lda_model.prepare(lda_Q2_corpus, bow_Q2, bow_vectorizer, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      16.812860 -87.325089       1        1  32.084340
0     -54.854027 -89.248886       2        1  27.351525
1     -57.463196 -17.704569       3        1  21.405598
3      14.010974 -15.732894       4        1  19.158537, topic_info=           Term       Freq      Total Category  logprob  loglift
154        hard   9.000000   9.000000  Default  30.0000  30.0000
414        work  25.000000  25.000000  Default  29.0000  29.0000
226     meeting   3.000000   3.000000  Default  28.0000  28.0000
119  experience   3.000000   3.000000  Default  27.0000  27.0000
1          able   6.000000   6.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
379       think   2.014025  13.620781   Topic4  -4.4443  -0.2590
404   virtually   1.405436   5.922538   Topic4  -4.8041   0.2140
1          able   1.400413   6.109671   Topic4  -4.8076   0.1793
248      office   1.399090   6.842940   Topic4  -4.8086   0.0650
76          day   1.399641   9.031331   Topic4  -4.8082  -0.2121

[248 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         1  0.163675        able
1         2  0.654700        able
1         4  0.163675        able
2         3  0.758341  absolutely
4         1  0.934982         add
...     ...       ...         ...
413       4  0.383336     without
414       1  0.192944        work
414       2  0.501654        work
414       3  0.192944        work
414       4  0.115766        work

[301 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 4])

## **Find a dominant topin in a answer**

Let's assign a dominant topic to each answer in our corpus. Weights in a word vector for a topic provide a measure of association for the word with the topic. If you sum weights for a particular topic across all words in a answer, you'll get the weight of that topic in the answer.

In [ ]:
lda_Q1_corpus_weights = lda_Q1_corpus.transform(bow_Q1)

Let's convert lda_Q1_corpus_weights into a nice-looking dataframe and have a look at the computed topic weights in documents:

In [ ]:
#array of answer "names" and topic "names" ("names" are just indecies)
ans_names = ["Ans_" + str(i) for i in range(len(NORM_Q1))]
topic_names = ["Topic_" + str(i) for i in range(4)]

#convert to dataframe
df_answer_topic = pd.DataFrame(np.round(lda_Q1_corpus_weights, 4), columns=topic_names, index=ans_names)
df_answer_topic.head(5)

,Topic_0,Topic_1,Topic_2,Topic_3
Ans_0,0.9430,0.0200,0.0189,0.0181
Ans_1,0.0288,0.9141,0.0291,0.0281
Ans_2,0.0188,0.9446,0.0186,0.0180
Ans_3,0.0096,0.9711,0.0097,0.0096
Ans_4,0.0281,0.9151,0.0289,0.0279


The topic with the highest weight in each document is a dominant topic. The weights across the 4 topics sum up to 1. Let's add a column that shows dominant topic for each document:

In [ ]:
#vector of indecies for columns with the highest value by each row in df_answer_topic
dominant_topic = np.argmax(df_answer_topic.values, axis=1)

#add dominant_topic as a column to df_document_topic
df_answer_topic['dominant_topic'] = dominant_topic
df_answer_topic.head(5)

,Topic_0,Topic_1,Topic_2,Topic_3,dominant_topic
Ans_0,0.9430,0.0200,0.0189,0.0181,0
Ans_1,0.0288,0.9141,0.0291,0.0281,1
Ans_2,0.0188,0.9446,0.0186,0.0180,1
Ans_3,0.0096,0.9711,0.0097,0.0096,1
Ans_4,0.0281,0.9151,0.0289,0.0279,1


## **Topic Meol Evaluation: Coherence Scores**

We will use a function CoherenceModel() from the gensim module

In [ ]:
!{sys.executable} -m pip install gensim
import gensim

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

The function CoherenceModel() needs as inputs:

1. Dictionary of the corpus
2. Corpus with each document represented as Bag-of-Words
3. An array of top words for each topic: we'll have top 20 words for each topic

We will now create those objects:

In [ ]:
#tokenizing the corpus
Q1_corpus_tokenized = [tokenize_text(NORM_Q1[ans_id]) for ans_id in range(len(NORM_Q1))]

#Dictionary of the corpus:
Q1_dictionary = Dictionary(Q1_corpus_tokenized)

#Bag-of-words representation for each document of the corpus:
Q1_corpus_bow = [Q1_dictionary.doc2bow(ans) for ans in Q1_corpus_tokenized]

#top 20 words for each topic (using the function defined in toolkit)
topic_topwords = get_topic_words(vectorizer = bow_vectorizer, lda_model = lda_Q1_corpus, n_words=20)

Now let's compute the coherence score for the model overall. We use one of the coherence metrics "u-mass" which measures semantic similarity of words in a topic, but there are other metrics as well.

In [ ]:
cm = CoherenceModel(topics=topic_topwords,
                    corpus = Q1_corpus_bow ,
                    dictionary = Q1_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 4))  # get coherence value

Coherence score for the model:  -6.2575


You can also see **coherence scores by topic**:

In [ ]:
print("Coherence score by topic (higher values are better): ", np.round(cm.get_coherence_per_topic(),4))

Coherence score by topic (higher values are better):  [-6.1508 -5.0146 -5.1128 -8.7517]
